In [23]:
from langchain_community.llms import Ollama
llm = Ollama(model = "mistral")

In [30]:
import os
os.environ.update({'OLLAMA_HOST': '0.0.0.0'})

In [33]:
llm.invoke('hi')

" Hello! How can I help you today? Is there something specific you would like to know or discuss? I'm here to answer any questions and provide information on a wide variety of topics. Let me know if you have any questions about science, technology, history, literature, mathematics, or anything else that comes to mind. If you just want to chat, we can do that too! What would you like to talk about?"